In [1]:
import torch
import time
from torch.autograd import Variable
from torch.distributions import Normal
from torch.distributions import Uniform
from sklearn.datasets import load_iris
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
class SimulatedAnnealing(torch.optim.Optimizer):
    def __init__(self, params, init_T, annealing_rate, max_iter):
        defaults = dict(init_T=init_T, annealing_rate=annealing_rate, max_iter=max_iter, T=init_T, total_iter=1)
        super(SimulatedAnnealing, self).__init__(params, defaults)
        
    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()
            
        dist = Normal(torch.tensor(0.0), torch.tensor(0.01))
        
        for group in self.param_groups:
            group['iter'] = 1
            while group['iter'] <= group['max_iter']:
                init_T = group['init_T']
                annealing_rate = group['annealing_rate']

                best_params = [p.clone() for p in group['params']]

                for p in group['params']:
                    bias = dist.sample(p.data.size())
                    p.data.add_(bias)

                new_loss = closure()
                if new_loss.item() <= loss.item():
                    for p, pbkp in zip(group['params'], best_params):
                        pbkp.data = p.data.clone()
                    loss = new_loss.clone()
                else:
                    acceptance_p = torch.exp((loss - new_loss)/group['T'])
                    if acceptance_p > Uniform(0, 1).sample(acceptance_p.size()):
                        for p, pbkp in zip(group['params'], best_params):
                            pbkp.data = p.data.clone()
                        loss = new_loss.clone()
                    else:
                        for p, pbkp in zip(group['params'], best_params):
                            p.data = pbkp.data.clone()
                    group['T'] = group['init_T'] * annealing_rate / group['total_iter']
                    group['iter'] += 1
                    group['total_iter'] += 1
            
        return loss

In [3]:
# Loading iris dataset using sklearn built-in feature
iris = load_iris()
x_data = iris.data
y_data = iris.target

In [4]:
# Using one-hot encoding to convert integers to categorical data
y_data = to_categorical(y_data)

In [5]:
# Converting from numpy data structure to pytorch's wrapper
x_data = Variable(torch.from_numpy(x_data))
y_data = Variable(torch.from_numpy(y_data))

In [6]:
# Building the neural network model
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.l1 = torch.nn.Linear(4, 12)
        self.l2 = torch.nn.Linear(12, 16)
        self.l3 = torch.nn.Linear(16, 3)
        self.sigmoid=torch.nn.Sigmoid()
        self.relu=torch.nn.ReLU()
        
    def forward(self,x):
        out1=self.relu(self.l1(x))
        out2=self.relu(self.l2(out1))
        y_pred=self.sigmoid(self.l3(out2))
        return y_pred

In [7]:
# Getting summary of a model
model = Model()
model

Model(
  (l1): Linear(in_features=4, out_features=12, bias=True)
  (l2): Linear(in_features=12, out_features=16, bias=True)
  (l3): Linear(in_features=16, out_features=3, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)

In [8]:
# Definining Binary Cross Entropy as a loss function
loss_f = torch.nn.BCELoss(reduction='mean')

In [9]:
# Setting up training parameters
batch_size=16
init_T=500
annealing_rate=0.85
max_iter = 100
epochs = 100

In [10]:
# Using Simulated Annealing as an optimizer
opt = SimulatedAnnealing(model.parameters(), init_T=init_T, annealing_rate=annealing_rate, max_iter=max_iter)

In [11]:
# Shuffling data
permutation = torch.randperm(x_data.size()[0])

In [12]:
# Training
start_time = time.time()
for epoch in range(epochs):
    for i in range(0, x_data.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        
        x, y = x_data[indices], y_data[indices]
        y_pred=model(x.float())
        
        def closure():
            y_pred = model(x.float())
            loss = loss_f(y_pred, y)
            opt.zero_grad()
            loss.backward()
            return loss
        
        loss = opt.step(closure)
        print (f'Epoch: {str(epoch)}, batch: {str(i)}, loss: {loss}')
duration = time.time() - start_time

Epoch: 0, batch: 0, loss: 0.6273958086967468
Epoch: 0, batch: 16, loss: 0.6949339509010315
Epoch: 0, batch: 32, loss: 0.6865144371986389
Epoch: 0, batch: 48, loss: 0.7181211113929749
Epoch: 0, batch: 64, loss: 1.1376310586929321
Epoch: 0, batch: 80, loss: 1.2469877004623413
Epoch: 0, batch: 96, loss: 0.7723749279975891
Epoch: 0, batch: 112, loss: 0.7093414664268494
Epoch: 0, batch: 128, loss: 0.9261996150016785
Epoch: 0, batch: 144, loss: 1.1008186340332031
Epoch: 1, batch: 0, loss: 1.3429183959960938
Epoch: 1, batch: 16, loss: 1.2605472803115845
Epoch: 1, batch: 32, loss: 1.8603216409683228
Epoch: 1, batch: 48, loss: 1.067273736000061
Epoch: 1, batch: 64, loss: 0.7019075751304626
Epoch: 1, batch: 80, loss: 0.7316166758537292
Epoch: 1, batch: 96, loss: 0.6628550887107849
Epoch: 1, batch: 112, loss: 0.8085906505584717
Epoch: 1, batch: 128, loss: 0.8517827391624451
Epoch: 1, batch: 144, loss: 0.9902262091636658
Epoch: 2, batch: 0, loss: 0.9680604934692383
Epoch: 2, batch: 16, loss: 0.951

Epoch: 17, batch: 64, loss: 0.08952922374010086
Epoch: 17, batch: 80, loss: 0.06975875049829483
Epoch: 17, batch: 96, loss: 0.173789843916893
Epoch: 17, batch: 112, loss: 0.16950251162052155
Epoch: 17, batch: 128, loss: 0.10189759731292725
Epoch: 17, batch: 144, loss: 0.15220047533512115
Epoch: 18, batch: 0, loss: 0.30893346667289734
Epoch: 18, batch: 16, loss: 0.191214919090271
Epoch: 18, batch: 32, loss: 0.13576357066631317
Epoch: 18, batch: 48, loss: 0.16105471551418304
Epoch: 18, batch: 64, loss: 0.1226775273680687
Epoch: 18, batch: 80, loss: 0.06565332412719727
Epoch: 18, batch: 96, loss: 0.11864554136991501
Epoch: 18, batch: 112, loss: 0.12543974816799164
Epoch: 18, batch: 128, loss: 0.19139541685581207
Epoch: 18, batch: 144, loss: 0.1215011402964592
Epoch: 19, batch: 0, loss: 0.24104177951812744
Epoch: 19, batch: 16, loss: 0.17305274307727814
Epoch: 19, batch: 32, loss: 0.11576201766729355
Epoch: 19, batch: 48, loss: 0.16019479930400848
Epoch: 19, batch: 64, loss: 0.181983754038

Epoch: 34, batch: 48, loss: 0.010052735917270184
Epoch: 34, batch: 64, loss: 0.052993178367614746
Epoch: 34, batch: 80, loss: 0.008125697262585163
Epoch: 34, batch: 96, loss: 0.002319388324394822
Epoch: 34, batch: 112, loss: 0.0023849455174058676
Epoch: 34, batch: 128, loss: 0.011403631418943405
Epoch: 34, batch: 144, loss: 3.057676076423377e-05
Epoch: 35, batch: 0, loss: 0.024468936026096344
Epoch: 35, batch: 16, loss: 0.002460331888869405
Epoch: 35, batch: 32, loss: 0.012793944217264652
Epoch: 35, batch: 48, loss: 0.007635436952114105
Epoch: 35, batch: 64, loss: 0.0373929925262928
Epoch: 35, batch: 80, loss: 0.0009613309521228075
Epoch: 35, batch: 96, loss: 0.006324369926005602
Epoch: 35, batch: 112, loss: 0.01243677455931902
Epoch: 35, batch: 128, loss: 0.003803860628977418
Epoch: 35, batch: 144, loss: 0.01966117136180401
Epoch: 36, batch: 0, loss: 0.38057318329811096
Epoch: 36, batch: 16, loss: 0.000455474597401917
Epoch: 36, batch: 32, loss: 0.006764108780771494
Epoch: 36, batch: 

Epoch: 51, batch: 0, loss: 0.07374989241361618
Epoch: 51, batch: 16, loss: 0.010139510966837406
Epoch: 51, batch: 32, loss: 0.002001106273382902
Epoch: 51, batch: 48, loss: 0.014534602873027325
Epoch: 51, batch: 64, loss: 0.004753883928060532
Epoch: 51, batch: 80, loss: 2.468697857693769e-06
Epoch: 51, batch: 96, loss: 0.004025258589535952
Epoch: 51, batch: 112, loss: 0.01652550883591175
Epoch: 51, batch: 128, loss: 0.007541893515735865
Epoch: 51, batch: 144, loss: 0.002888481132686138
Epoch: 52, batch: 0, loss: 0.0440739206969738
Epoch: 52, batch: 16, loss: 0.004005701746791601
Epoch: 52, batch: 32, loss: 0.0016438947059214115
Epoch: 52, batch: 48, loss: 0.006685614585876465
Epoch: 52, batch: 64, loss: 0.0521625317633152
Epoch: 52, batch: 80, loss: 7.456103048752993e-05
Epoch: 52, batch: 96, loss: 0.0061246235854923725
Epoch: 52, batch: 112, loss: 0.006831422448158264
Epoch: 52, batch: 128, loss: 0.016910044476389885
Epoch: 52, batch: 144, loss: 4.834122955799103e-05
Epoch: 53, batch:

Epoch: 67, batch: 96, loss: 0.002584283472970128
Epoch: 67, batch: 112, loss: 0.04845608398318291
Epoch: 67, batch: 128, loss: 0.0008624849724583328
Epoch: 67, batch: 144, loss: 2.7087523903901456e-06
Epoch: 68, batch: 0, loss: 0.06599237769842148
Epoch: 68, batch: 16, loss: 0.019363772124052048
Epoch: 68, batch: 32, loss: 0.0005404512048698962
Epoch: 68, batch: 48, loss: 0.013627261854708195
Epoch: 68, batch: 64, loss: 0.03355328366160393
Epoch: 68, batch: 80, loss: 4.052262283948949e-06
Epoch: 68, batch: 96, loss: 1.1889062989212107e-05
Epoch: 68, batch: 112, loss: 0.006795145105570555
Epoch: 68, batch: 128, loss: 0.0006931281532160938
Epoch: 68, batch: 144, loss: 0.0044043711386621
Epoch: 69, batch: 0, loss: 0.1699928492307663
Epoch: 69, batch: 16, loss: 4.704264210886322e-05
Epoch: 69, batch: 32, loss: 1.1759831295421463e-06
Epoch: 69, batch: 48, loss: 0.005685858894139528
Epoch: 69, batch: 64, loss: 0.004574497230350971
Epoch: 69, batch: 80, loss: 1.3535264997699414e-07
Epoch: 69,

Epoch: 84, batch: 16, loss: 7.393774285446852e-05
Epoch: 84, batch: 32, loss: 0.0002905646397266537
Epoch: 84, batch: 48, loss: 0.001844971557147801
Epoch: 84, batch: 64, loss: 0.021623888984322548
Epoch: 84, batch: 80, loss: 0.0003875151742249727
Epoch: 84, batch: 96, loss: 0.0016388093354180455
Epoch: 84, batch: 112, loss: 0.0007673152722418308
Epoch: 84, batch: 128, loss: 0.0003892310487572104
Epoch: 84, batch: 144, loss: 7.285017034064367e-08
Epoch: 85, batch: 0, loss: 0.8546666502952576
Epoch: 85, batch: 16, loss: 1.266602538407824e-07
Epoch: 85, batch: 32, loss: 0.0006446473998948932
Epoch: 85, batch: 48, loss: 0.004210053477436304
Epoch: 85, batch: 64, loss: 0.005364192184060812
Epoch: 85, batch: 80, loss: 6.854647267573455e-07
Epoch: 85, batch: 96, loss: 1.6600126400589943e-05
Epoch: 85, batch: 112, loss: 0.0052200015634298325
Epoch: 85, batch: 128, loss: 8.465551218250766e-06
Epoch: 85, batch: 144, loss: 0.008830131962895393
Epoch: 86, batch: 0, loss: 2.7041914463043213
Epoch:

In [13]:
# Printing time (in seconds) required for training
duration

128.63637447357178

In [14]:
model = Model()

In [15]:
# Now changing the optimizer to Gradient Descent
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [16]:
# Training
start_time = time.time()
for epoch in range(epochs):
    for i in range(0, x_data.size()[0], batch_size):
        indices = permutation[i:i+batch_size]
        x, y = x_data[indices], y_data[indices]
        opt.zero_grad()
        y_pred=model(x.float())
        loss = loss_f(y_pred, y)
        loss.backward()
        opt.step()
        print (f'Epoch: {str(epoch)}, batch: {str(i)}, loss: {loss}')
duration = time.time() - start_time

Epoch: 0, batch: 0, loss: 0.7108926773071289
Epoch: 0, batch: 16, loss: 0.7198290824890137
Epoch: 0, batch: 32, loss: 0.6937561631202698
Epoch: 0, batch: 48, loss: 0.6946389079093933
Epoch: 0, batch: 64, loss: 0.7167690396308899
Epoch: 0, batch: 80, loss: 0.6990072131156921
Epoch: 0, batch: 96, loss: 0.7288073897361755
Epoch: 0, batch: 112, loss: 0.7033264636993408
Epoch: 0, batch: 128, loss: 0.6821077466011047
Epoch: 0, batch: 144, loss: 0.6250627040863037
Epoch: 1, batch: 0, loss: 0.6454740166664124
Epoch: 1, batch: 16, loss: 0.6295414566993713
Epoch: 1, batch: 32, loss: 0.636696457862854
Epoch: 1, batch: 48, loss: 0.622603714466095
Epoch: 1, batch: 64, loss: 0.6523577570915222
Epoch: 1, batch: 80, loss: 0.6703734397888184
Epoch: 1, batch: 96, loss: 0.6608941555023193
Epoch: 1, batch: 112, loss: 0.6487292647361755
Epoch: 1, batch: 128, loss: 0.6278406977653503
Epoch: 1, batch: 144, loss: 0.5510778427124023
Epoch: 2, batch: 0, loss: 0.6027626991271973
Epoch: 2, batch: 16, loss: 0.5727

Epoch: 17, batch: 64, loss: 0.3002234995365143
Epoch: 17, batch: 80, loss: 0.2388046532869339
Epoch: 17, batch: 96, loss: 0.3312332332134247
Epoch: 17, batch: 112, loss: 0.3095703125
Epoch: 17, batch: 128, loss: 0.272870808839798
Epoch: 17, batch: 144, loss: 0.2619389295578003
Epoch: 18, batch: 0, loss: 0.2524125277996063
Epoch: 18, batch: 16, loss: 0.29941248893737793
Epoch: 18, batch: 32, loss: 0.2209010273218155
Epoch: 18, batch: 48, loss: 0.2482268214225769
Epoch: 18, batch: 64, loss: 0.29544419050216675
Epoch: 18, batch: 80, loss: 0.23576509952545166
Epoch: 18, batch: 96, loss: 0.3253612816333771
Epoch: 18, batch: 112, loss: 0.3049931228160858
Epoch: 18, batch: 128, loss: 0.2666374742984772
Epoch: 18, batch: 144, loss: 0.2573660910129547
Epoch: 19, batch: 0, loss: 0.24874861538410187
Epoch: 19, batch: 16, loss: 0.29550373554229736
Epoch: 19, batch: 32, loss: 0.21654053032398224
Epoch: 19, batch: 48, loss: 0.24320296943187714
Epoch: 19, batch: 64, loss: 0.2907392680644989
Epoch: 19

Epoch: 34, batch: 80, loss: 0.18126119673252106
Epoch: 34, batch: 96, loss: 0.23725564777851105
Epoch: 34, batch: 112, loss: 0.2428024560213089
Epoch: 34, batch: 128, loss: 0.17989321053028107
Epoch: 34, batch: 144, loss: 0.20449398458003998
Epoch: 35, batch: 0, loss: 0.19769692420959473
Epoch: 35, batch: 16, loss: 0.23283173143863678
Epoch: 35, batch: 32, loss: 0.14820705354213715
Epoch: 35, batch: 48, loss: 0.1656017154455185
Epoch: 35, batch: 64, loss: 0.22923563420772552
Epoch: 35, batch: 80, loss: 0.17819558084011078
Epoch: 35, batch: 96, loss: 0.2322722226381302
Epoch: 35, batch: 112, loss: 0.23805153369903564
Epoch: 35, batch: 128, loss: 0.17292232811450958
Epoch: 35, batch: 144, loss: 0.19820445775985718
Epoch: 36, batch: 0, loss: 0.19231124222278595
Epoch: 36, batch: 16, loss: 0.2270582914352417
Epoch: 36, batch: 32, loss: 0.14183813333511353
Epoch: 36, batch: 48, loss: 0.15855886042118073
Epoch: 36, batch: 64, loss: 0.22439365088939667
Epoch: 36, batch: 80, loss: 0.1729081124

Epoch: 51, batch: 80, loss: 0.10088757425546646
Epoch: 51, batch: 96, loss: 0.11071667820215225
Epoch: 51, batch: 112, loss: 0.1529018133878708
Epoch: 51, batch: 128, loss: 0.06138971820473671
Epoch: 51, batch: 144, loss: 0.15354350209236145
Epoch: 52, batch: 0, loss: 0.1277519017457962
Epoch: 52, batch: 16, loss: 0.16034136712551117
Epoch: 52, batch: 32, loss: 0.05385814607143402
Epoch: 52, batch: 48, loss: 0.05700197443366051
Epoch: 52, batch: 64, loss: 0.1563565880060196
Epoch: 52, batch: 80, loss: 0.0963074341416359
Epoch: 52, batch: 96, loss: 0.10485974699258804
Epoch: 52, batch: 112, loss: 0.1504429429769516
Epoch: 52, batch: 128, loss: 0.05799200013279915
Epoch: 52, batch: 144, loss: 0.15008698403835297
Epoch: 53, batch: 0, loss: 0.12604601681232452
Epoch: 53, batch: 16, loss: 0.15638063848018646
Epoch: 53, batch: 32, loss: 0.051465705037117004
Epoch: 53, batch: 48, loss: 0.05388980731368065
Epoch: 53, batch: 64, loss: 0.1529160887002945
Epoch: 53, batch: 80, loss: 0.09167846292

Epoch: 68, batch: 80, loss: 0.041696697473526
Epoch: 68, batch: 96, loss: 0.045988429337739944
Epoch: 68, batch: 112, loss: 0.11435768008232117
Epoch: 68, batch: 128, loss: 0.034277234226465225
Epoch: 68, batch: 144, loss: 0.10279885679483414
Epoch: 69, batch: 0, loss: 0.12006640434265137
Epoch: 69, batch: 16, loss: 0.09638618677854538
Epoch: 69, batch: 32, loss: 0.033248405903577805
Epoch: 69, batch: 48, loss: 0.029288649559020996
Epoch: 69, batch: 64, loss: 0.11312706023454666
Epoch: 69, batch: 80, loss: 0.03978591412305832
Epoch: 69, batch: 96, loss: 0.044212788343429565
Epoch: 69, batch: 112, loss: 0.1126970574259758
Epoch: 69, batch: 128, loss: 0.0337575264275074
Epoch: 69, batch: 144, loss: 0.10074342042207718
Epoch: 70, batch: 0, loss: 0.12044215947389603
Epoch: 70, batch: 16, loss: 0.0943254604935646
Epoch: 70, batch: 32, loss: 0.03273046389222145
Epoch: 70, batch: 48, loss: 0.028542963787913322
Epoch: 70, batch: 64, loss: 0.11179220676422119
Epoch: 70, batch: 80, loss: 0.03803

Epoch: 85, batch: 64, loss: 0.09589975327253342
Epoch: 85, batch: 80, loss: 0.021539121866226196
Epoch: 85, batch: 96, loss: 0.028302786871790886
Epoch: 85, batch: 112, loss: 0.0947037860751152
Epoch: 85, batch: 128, loss: 0.029616350308060646
Epoch: 85, batch: 144, loss: 0.07764586806297302
Epoch: 86, batch: 0, loss: 0.1283363252878189
Epoch: 86, batch: 16, loss: 0.07031819969415665
Epoch: 86, batch: 32, loss: 0.027609186246991158
Epoch: 86, batch: 48, loss: 0.021073563024401665
Epoch: 86, batch: 64, loss: 0.09501703828573227
Epoch: 86, batch: 80, loss: 0.020736603066325188
Epoch: 86, batch: 96, loss: 0.027683166787028313
Epoch: 86, batch: 112, loss: 0.09366672486066818
Epoch: 86, batch: 128, loss: 0.02954353392124176
Epoch: 86, batch: 144, loss: 0.0763256847858429
Epoch: 87, batch: 0, loss: 0.1287267804145813
Epoch: 87, batch: 16, loss: 0.06900135427713394
Epoch: 87, batch: 32, loss: 0.027327360585331917
Epoch: 87, batch: 48, loss: 0.020726824179291725
Epoch: 87, batch: 64, loss: 0.0

In [17]:
duration

2.365001916885376